In [3]:
using Pkg

In [14]:
ENV["CPLEX_STUDIO_BINARIES"] = "C:\\Program Files\\IBM\\ILOG\\CPLEX_Studio1210\\cplex\\bin\\x64_win64"

"C:\\Program Files\\IBM\\ILOG\\CPLEX_Studio1210\\cplex\\bin\\x64_win64"

In [15]:
Pkg.build("CPLEX")

   Building CPLEX → `C:\Users\Geoffroy Leconte\.julia\packages\CPLEX\Hxpuk\deps\build.log`


In [28]:
using CPLEX
using QPSReader
using QuadraticModels
using NLPModels
using SolverTools
using SolverBenchmark
using DataFrames
using LinearAlgebra
using SparseArrays

In [29]:
function createQuadraticModel(qpdata)
    # probleme du point initial
    #x0 = init_x0(qpdata.lvar, qpdata.uvar)
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0)

end

function display_results_CPlex(xpress_model)
    # show results
    sol = CPLEX.get_solution(model_cplex)
    println("soln = $(sol)")

    objv = CPLEX.get_objval(model_cplex)
    println("objv = $(objv)")
end
    

function optimizeCPlex(QM)
    SM = SlackModel(QM)
    
    env = CPLEX.Env()
    # access variables in original problem, not presolved
    #CPLEX.set_param!(env, "CPX_PARAM_MIPCBREDLP", 0) 
    #CPLEX.set_param!(env, "CPXPARAM_Barrier_Algorithm", 0)
    
    CPLEX.set_param!(env, "CPXPARAM_ScreenOutput", 1)  # Enable output
    CPLEX.set_param!(env, "CPXPARAM_TimeLimit", 3600)  # Time limit
    CPLEX.set_param!(env, "CPXPARAM_Threads", 1)  # Single thread
    CPLEX.set_param!(env, "CPXPARAM_SolutionType", 2)  # No crossover
    CPLEX.set_param!(env, "CPXPARAM_LPMethod", 4)  # Use barrier
    CPLEX.set_param!(env, "CPXPARAM_Preprocessing_Presolve", 0)  # No presolve
    CPLEX.set_param!(env, "CPXPARAM_Read_Scale", -1) # no scaling
    
    Aeq = jac(SM, SM.meta.x0)
    beq = SM.meta.lcon
    H = hess(SM, zeros(length(SM.meta.x0)))
    H = sparse(Symmetric(H, :L))
    f = grad(SM, zeros(length(SM.meta.x0)))
    n,m = size(Aeq)    
    
    model = CPLEX.cplex_model(env; f = f,
                Aeq = Aeq, beq = beq, 
                lb = SM.meta.lvar, ub = SM.meta.uvar)
    # H = ... if quadratic problem
     # run optimization
    t = @timed begin
        CPLEX.optimize!(model)
    end
    
    #println(CPLEX.c_api_solninfo(model))
    x = CPLEX.get_solution(model)
    
    y = CPLEX.get_constr_duals(model)
    
    s = CPLEX.get_reduced_costs(model)
    
    if model.terminator == [0]
        status = :acceptable
    else
        status = :unknown
    end
    #println(env)
    #println(CPLEX.get_status(model))
    
    stats = GenericExecutionStats(status, SM, solution = x,
                                  objective = CPLEX.get_objval(model), 
                                  primal_feas = norm(Aeq * x - beq, Inf),
                                  dual_feas = norm(Aeq' * y + s - f, Inf),
                                  iter = Int64(CPLEX.c_api_getitcnt(model)),
                                  multipliers = y,
                                  elapsed_time = t[2])
    
    return stats
end

function optimizeCPlex(qpdata::QPSData)
    return optimizeCPlex(createQuadraticModel(qpdata))
end

optimizeCPlex (generic function with 2 methods)

In [30]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib"
afiro = string(path_pb, "\\AFIRO.SIF")
qpdata2 = readqps(afiro);

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [31]:
stats2 = optimizeCPlex(qpdata2)
println(stats2)

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_Scale                              -1
CPXPARAM_LPMethod                                4
CPXPARAM_Threads                                 1
CPXPARAM_SolutionType                            2
CPXPARAM_TimeLimit                               3600
Tried aggregator 0 times.
Reduced presolve eliminated 3 rows and 19 columns.
Reduced LP has 24 rows, 32 columns, and 79 nonzeros.
Presolve time = 0.00 sec. (0.03 ticks)
Parallel mode: none, using 1 thread for barrier
Number of nonzeros in lower triangle of A*A' = 55
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.01 ticks)
Summary statistics for Cholesky factor:
  Rows in Factor            = 24
  Integer space required    = 30
  Total non-zeros in factor = 223
  Total FP ops to factor    = 2913
 Itn      Primal Obj        Dual Obj  Prim Inf Upper Inf  Dual Inf Inf Ratio
   0   1.21951

In [32]:
# problem 3   kb2    obj  -1.7499001299E+03
kb2 = string(path_pb, "\\25FV47.SIF")
qpdata3 = readqps(kb2)
stats3 =  optimizeCPlex(qpdata3)
println(stats3)

┌ Info: Using '25FV47' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'R0000' as objective (l. 21)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350


Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_Scale                              -1
CPXPARAM_LPMethod                                4
CPXPARAM_Threads                                 1
CPXPARAM_SolutionType                            2
CPXPARAM_TimeLimit                               3600
Tried aggregator 0 times.
Reduced presolve eliminated 50 rows and 340 columns.
Reduced LP has 771 rows, 1536 columns, and 10116 nonzeros.
Presolve time = 0.02 sec. (2.47 ticks)
Parallel mode: none, using 1 thread for barrier
Number of nonzeros in lower triangle of A*A' = 10653
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (1.10 ticks)
Summary statistics for Cholesky factor:
  Rows in Factor            = 771
  Integer space required    = 4722
  Total non-zeros in factor = 35724
  Total FP ops to factor    = 2590312
 Itn      Primal Obj        Dual Obj  Prim Inf Upper Inf  Dual Inf Inf R

┌ Info: Using '.00001' as RHS (l. 6793)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
